In [1]:
import os

In [2]:
output_path = os.path.abspath(os.path.join('__file__',
                                    '../..',
                                    'data',
                                    'Catalogue',
                                    'Raw'))
print(output_path)

C:\Users\fuwei\Desktop\GWASReview\GWASReview-master\data\Catalogue\Raw


In [8]:
import sys
!{sys.executable} -m pip install numpy 
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install requests_ftp

### Download the raw GWAS data from the ebi website. 
Here, the function download_cat need to be revoked, which is located in **Module: loaddata.py**

In [9]:
import requests
import requests_ftp

def download_cat(path, ebi_download):
    """ download the data from the ebi main site and ftp"""
    r = requests.get(ebi_download + 'studies_alternative')
    with open(os.path.join(path, 'Cat_Stud.tsv'), 'wb') as tsvfile:
        tsvfile.write(r.content)
    r = requests.get(ebi_download + 'ancestry')
    with open(os.path.join(path, 'Cat_Anc.tsv'), 'wb') as tsvfile:
        tsvfile.write(r.content)H
    r = requests.get(ebi_download + 'full')
    with open(os.path.join(path, 'Cat_Full.tsv'), 'wb') as tsvfile:
        tsvfile.write(r.content)
    requests_ftp.monkeypatch_session()
    s = requests.Session()
    ftpsite = 'ftp://ftp.ebi.ac.uk/'
    subdom = '/pub/databases/gwas/releases/latest/'
    file = 'gwas-efo-trait-mappings.tsv'
    r = s.get(ftpsite+subdom+file)
    with open(os.path.join(path, 'Cat_Map.tsv'), 'wb') as tsvfile:
        tsvfile.write(r.content)

ebi_download = 'https://www.ebi.ac.uk/gwas/api/search/downloads/'
download_cat(output_path, ebi_download)

### Lets link the PUBMEDID variable to the PUBMED AP.
This will get a series of datasets from that using the support functions written in PubMed.py. Note: collective corresponds mostly consortia and study groups.
* function load_gwas_cat() in **module: ** need to be revoked.

In [ ]:
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip uninstall numpy
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install biopython
!{sys.executable} -m pip install Bio

In [38]:
import pandas as pd
import numpy
from Bio import Entrez

def load_gwas_cat():
    """A function which cleans and loads all the main GWAS Catalog files into
    the notebook workspace. It renames a couple of fields, and creates
    one groupedby.sum for return
    """
    Cat_Stud = pd.read_csv(os.path.abspath(
                           os.path.join('__file__',
                                        '../..',
                                        'data',
                                        'Catalogue',
                                        'Raw',
                                        'Cat_Stud.tsv')),
                           header=0, sep='\t', encoding='utf-8',
                           index_col=False)
    Cat_Stud.fillna('N/A', inplace=True)
    Cat_Anc = pd.read_csv(os.path.abspath(
                          os.path.join('__file__', '../..',
                                       'data',
                                       'Catalogue',
                                       'Raw',
                                       'Cat_Anc.tsv')),
                          header=0, sep='\t', encoding='utf-8',
                          index_col=False)
    Cat_Anc.rename(columns={'BROAD ANCESTRAL CATEGORY': 'BROAD ANCESTRAL',
                            'NUMBER OF INDIVDUALS': 'N'}, inplace=True)
    Cat_Anc = Cat_Anc[~Cat_Anc['BROAD ANCESTRAL'].isnull()]
    Cat_Anc.columns = Cat_Anc.columns.str.replace('ACCCESSION', 'ACCESSION')
    Cat_Anc_byN = Cat_Anc[['STUDY ACCESSION', 'N',
                           'DATE']].groupby(by='STUDY ACCESSION').sum()
    Cat_Anc_byN = Cat_Anc_byN.reset_index()
    Cat_Anc_byN = pd.merge(Cat_Anc_byN, Cat_Stud[[
        'STUDY ACCESSION', 'DATE']], how='left', on='STUDY ACCESSION')
    cleaner_broad = pd.read_csv(os.path.abspath(
        os.path.join('__file__',
                     '../..',
                     'data',
                     'Support',
                     'dict_replacer_broad.tsv')),
        sep='\t', header=0, index_col=False)
    Cat_Anc = pd.merge(Cat_Anc, cleaner_broad, how='left',
                       on='BROAD ANCESTRAL')
    Cat_Anc['Dates'] = [pd.to_datetime(d) for d in Cat_Anc['DATE']]
    Cat_Anc['N'] = pd.to_numeric(Cat_Anc['N'], errors='coerce')
    Cat_Anc = Cat_Anc[Cat_Anc['N'].notnull()]
    Cat_Anc['N'] = Cat_Anc['N'].astype(int)
    Cat_Anc = Cat_Anc.sort_values(by='Dates')
    Cat_Anc['Broader']
    Cat_Anc['Broader'] = Cat_Anc['Broader'].str.replace(
        'African American or Afro-Caribbean', 'African Am./Caribbean')
    Cat_Anc['Broader'] = Cat_Anc['Broader'].str.replace(
        'Hispanic or Latin American', 'Hispanic/Latin American')
    Cat_Full = pd.read_csv(os.path.abspath(os.path.join('__file__',
                                                        '../..',
                                                        'data',
                                                        'Catalogue',
                                                        'Raw',
                                                        'Cat_Full.tsv')),
                           header=0, sep='\t', encoding='utf-8',
                           index_col=False, low_memory=False)

    Cat_Anc.to_csv(os.path.abspath(
        os.path.join('__file__',
                     '../..',
                     'data',
                     'Catalogue',
                     'Synthetic',
                     'Cat_Anc_withBroader.tsv')),
                   sep='\t', index=False)
    return Cat_Stud, Cat_Anc, Cat_Anc_byN, Cat_Full

ModuleNotFoundError: No module named 'Bio'

In [17]:
Cat_Studies, Cat_Ancestry, Cat_Ancestry_groupedbyN, Cat_Full = load_gwas_cat()
id_list = set(Cat_Studies['PUBMEDID'].astype(str).tolist())
Entrez.email = 'your@email.com'
papers = Entrez.read(Entrez.efetch(db='pubmed',retmode='xml', id=','.join(id_list)))
build_collective(papers)
build_author(papers)
build_funder(papers)
build_abstract(papers)
build_citation(id_list,Entrez.email)

NameError: name 'Entrez' is not defined